<a href="https://colab.research.google.com/github/DeependraChaddha/RAG_Projects/blob/main/RAG_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SET UP ENVIRONMENT

PACKAGES

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain cohere

SET UP LANGSMITH

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']=##YOUR API KEY

OPENAI API KEY

In [ ]:
os.environ['OPENAI_API_KEY']=##YOUR API KEY
os.environ['COHERE_API_KEY']=##YOUR API KEY

##RE-RANKING

RE RANKING RANKS AND FILTER/COMPRESS DOCUMENTS BASED ON RELEVANCE

In [ ]:
###INDEXING###

#LOAD BLOG
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
        class_=("post-content","post-title", "post-header")
      )
    )
  )
blog_docs=loader.load()

#SPLIT
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

#MAKE SPLIT
splits=text_splitter.split_documents(blog_docs)

#INDEX
from langchain_openai import OpenAIEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain.vectorstores import Chroma
vectorstore=Chroma.from_documents(documents=splits,
                                  #embedding=CohereEmbeddings()
                                  embedding=OpenAIEmbeddings())
#RETRIEVER
retriever=vectorstore.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate

#RAG FUSION

template="""You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
#MAKE PROMPT FROM TEMPLATE
prompt_rag_fusion=Chat_Prompt_Template.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParsers
from langchain_openai import ChatOpenAI

generate_queries=(prompt_rag_fusion #make prompt
                  | ChatOpenAI(temperature=0) #put through llm
                  | StrOutputParsers()#parse output
                  |(lambda x:x.split("\n")))#split output lines

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list],
                           k=60):
  #RECIPROCAL RANK FUSION SAME AS BEFORE
  fused_scores={}

  #2. Iterate through each list of documents
  for docs in results:
    #2.1. iterating through each document in a list
    for rank, doc in enumerate(docs):
      #2.1.1. convert each doc to string (assuming docs can be serialized to JSON)
      doc_str=dumps(doc)
      #2.1.2. check if doc_str in not already present in fused_scores(to avoid repitition), if its not, then add with initia score=0
      if doc_str not in fused_scores:
        fused_scores[doc_str]=0
      #2.1.3. Retrieve the current score of the document(whether already present or just added)
      previous_score=fused_scores[doc_str]
      #2.1.4. Update score using RRF formula
      fused_scores+= 1/(rank + k)
  #3. Sort the documents based on rank in descending order
  reranked_results=[(loads(doc),score) for doc,score in sorted(fusion_scores.items().key= lambda x:x[1], reverse= True)]#Sorts fusion_scores dictionary according to score which is mentioned as the key and loads the doc and score into a list of tuples

  #4. Return the raranked list
  return reranked_results

question="What is task decomposition for LLM agents?"

#MAKING ENTIRE CHAIN
retrieval_chain_rag_fusion=generate_queries | retriever.map() | reciprocal_rank_fusion
docs=retrieval_chain_rag_fusion.invoke({"question":question})
len(docs)

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassThrough

#RAG
template="""Answer the following question based on this context:

{context}

Question: {question}
"""

prompt= Chat_Prompt_Template.from_template(template)

llm=ChatOpenAI(temperature=0)

final_rag_chain=(
    {"context":retrieval_chain_rag_fusion,
     "question":itemgetter("question")}
    |prompt
    |llm
    |StrOutputParser()
)

final_rag_chain.invoke(
    {"question":question}

)

##USE COHERE RERANK

Reranking, as the word suggests ranks the documents again after the first ranking, so that we get better ordering and better query results

In [ ]:
from langchain_community.llms import Cohere
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [ ]:
from langchain.retrievers.document_compressors import CohereRerank

retriever =vectorstore.as_retriever(search_kwargs={"k":10})

#RERANK
compressor=CohereRerank()
compression_retriever=ContextualCompressionRetriever(base_compressor=compressor,
                                                      base_retriever=retriever)
compressed_docs=compression_retriever.get_relevant_documents(question)